In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn.linear_model as linear_model
import datetime as dt
from datetime import timedelta

In [2]:
data1 = pd.read_csv("../../Project/data/201804-citibike-tripdata.csv")
data2 = pd.read_csv("../../Project/data/201805-citibike-tripdata.csv")
data3 = pd.read_csv("../../Project/data/201806-citibike-tripdata.csv")

In [33]:
data_eval = pd.read_csv("../../Project/data/201807-citibike-tripdata.csv")
data_test = pd.read_csv("../../Project/data/201808-citibike-tripdata.csv")

In [17]:
data = pd.concat((data1,data2,data3))

In [35]:
# Assumption functions

def get_duration(station1, station2):
    return 0

def user_end_distribution(data):
    # Station id and outflow percentage
    outflow_distribution = {'id': 0}
    return distribution

# get data from data frame
# return a list of all data
# start_time_min/max must be datetime objects
def get_unlock_data(start_time_min, start_time_max, start_station, full_data):
    if type(start_time_min) != dt.datetime or type(start_time_max) != dt.datetime:
        raise TypeError('Not datetime object.')
    data_for_station = full_data[full_data['start_station_id'] == start_station]
    data_list = data_for_station[data_for_station['start_time'] > start_time_min]
    data_list = data_for_station[data_for_station['start_time'] < start_time_max]
    return data_list

# subtract duration(minutes) from time correclty
def subtract_time(time, input_minutes):
    if type(time) != dt.datetime:
        raise TypeError('Not datetime object.')
    datetime_minutes = timedelta(minutes = input_minutes)
    adjusted_minutes = time - datetime_minutes
    return adjusted_minutes

# add duration(minutes) to time correclty
def add_time(time, input_minutes):
    if type(time) != dt.datetime:
        raise TypeError('Not datetime object.')
    datetime_minutes = timedelta(minutes = input_minutes)
    adjusted_minutes = time + datetime_minutes
    return adjusted_minutes

# Compare time
def smaller_than_time(time1, time2):
    if type(time1) != dt.datetime or type(time2) != dt.datetime:
        raise TypeError('Not datetime object.')
    return time1 < time2

def get_historical_inflow(start_station, end_station, end_time):
    ##
    return 0

In [36]:
# all_stations: a list of all stations
# interval: mins (+- 10 mins)
def calculate_inflow(predTime, curTime, end_station, all_stations, interval):
    ## look up how to calculate time
    # Calculate distribution for all stations
    inflow = 0
    for start_station in all_stations:
        trip_duration = get_duration(start_station, end_station)
        start_time = subtract_time(predTime, trip_duration)
        # use historical data if now is earlier than start time:
        if smaller_than_time(predTime, start_time):
            inflow += get_historical_inflow(start_station, end_station, predTime)
        else:
            unlock_list = get_unlock_data(subtract_time(start_time, interval / 2), 
                                          add_time(start_time, interval / 2),
                                          start_station)
            # add inflow for every user from the same start station
            for unlocked_user in unlock_list:
                inflow += user_end_distribution(unlocked_user)[end_station]
    return inflow

In [37]:
class Point:
    
    def __init__(self, x, y, swap = True):
        if swap:
            self.x = y
            self.y = x
        else:
            self.x = x
            self.y = y
        
class Line:
    
    def __init__(self, p1, p2):
        self.p1 = p1
        self.p2 = p2
        self.a = (p1.y - p2.y) / (p1.x - p2.x)
        self.b = p1.y - self.a * p1.x
        
    def getY(self, x):
        return self.a * x + self.b
        
    def getX(self, y):
        return (y - self.b) / self.a
    
    def above(self, input_point):
        return input_point.y >= self.getY(input_point.x)
    
    def below(self, input_point):
        return input_point.y < self.getY(input_point.x)
    
    def right(self, input_point):
        return input_point.x >= self.getX(input_point.y)
        
    def left(self, input_point):
        return input_point.x < self.getX(input_point.y)
    
class Region:
    
    def __init__(self, line_left, line_right, line_down, line_up, num):
        self.line_left = line_left
        self.line_right = line_right
        self.line_up = line_up
        self.line_down = line_down
        self.num = num
        
    def check_inside(self, point):
        return self.line_left.right(point) and \
               self.line_right.left(point) and \
               self.line_up.below(point) and \
               self.line_down.above(point)
    
    def get_num(self):
        return self.num

In [38]:
line_v1 = Line(Point(40.822042, -73.960223), Point(40.733240, -74.019613))
line_v2 = Line(Point(40.815704, -73.947402), Point(40.706711, -74.024827))
line_v3 = Line(Point(40.811526, -73.938080), Point(40.701839, -74.017955))
line_v4 = Line(Point(40.804877, -73.923991), Point(40.696625, -74.004297))
line_v5 = Line(Point(40.729934, -73.961664), Point(40.705813, -73.980607))

line_h11 = Line(Point(40.822042, -73.960223), Point(40.804877, -73.923991))
line_h10 = Line(Point(40.805881, -73.970669), Point(40.790774, -73.932337))
line_h9 = Line(Point(40.795345, -73.978905), Point(40.779025, -73.940987))
line_h8 = Line(Point(40.783599, -73.987055), Point(40.768406, -73.949353))
line_h7 = Line(Point(40.773192, -73.994511), Point(40.758769, -73.958902))
line_h6 = Line(Point(40.762348, -74.002173), Point(40.747673, -73.967231))
line_h5 = Line(Point(40.750092, -74.010230), Point(40.729934, -73.961664))
line_h4 = Line(Point(40.743822, -74.015333), Point(40.725047, -73.965458))
line_h3 = Line(Point(40.733240, -74.019613), Point(40.718461, -73.972046))
line_h2 = Line(Point(40.721485, -74.015598), Point(40.705813, -73.980607))
line_h1 = Line(Point(40.706711, -74.024827), Point(40.696625, -74.004297))

In [39]:
regions = []

regions.append(Region(line_v2, line_v3, line_h1, line_h2, 1))
regions.append(Region(line_v3, line_v4, line_h1, line_h2, 2))
regions.append(Region(line_v2, line_v3, line_h2, line_h3, 3))
regions.append(Region(line_v3, line_v4, line_h2, line_h3, 4))
regions.append(Region(line_v4, line_v5, line_h2, line_h3, 5))

regions.append(Region(line_v1, line_v2, line_h3, line_h4, 6))
regions.append(Region(line_v2, line_v3, line_h3, line_h4, 7))
regions.append(Region(line_v3, line_v4, line_h3, line_h4, 8))
regions.append(Region(line_v4, line_v5, line_h3, line_h4, 9))

regions.append(Region(line_v1, line_v2, line_h4, line_h5, 10))
regions.append(Region(line_v2, line_v3, line_h4, line_h5, 11))
regions.append(Region(line_v3, line_v4, line_h4, line_h5, 12))
regions.append(Region(line_v4, line_v5, line_h4, line_h5, 13))

regions.append(Region(line_v1, line_v2, line_h5, line_h6, 14))
regions.append(Region(line_v2, line_v3, line_h5, line_h6, 15))
regions.append(Region(line_v3, line_v4, line_h5, line_h6, 16))

regions.append(Region(line_v1, line_v2, line_h6, line_h7, 17))
regions.append(Region(line_v2, line_v3, line_h6, line_h7, 18))
regions.append(Region(line_v3, line_v4, line_h6, line_h7, 19))

regions.append(Region(line_v1, line_v2, line_h7, line_h8, 20))
regions.append(Region(line_v2, line_v3, line_h7, line_h8, 21))
regions.append(Region(line_v3, line_v4, line_h7, line_h8, 22))

regions.append(Region(line_v1, line_v2, line_h8, line_h9, 23))
regions.append(Region(line_v2, line_v3, line_h8, line_h9, 24))
regions.append(Region(line_v3, line_v4, line_h8, line_h9, 25))

regions.append(Region(line_v1, line_v2, line_h9, line_h10, 26))
regions.append(Region(line_v2, line_v3, line_h9, line_h10, 27))
regions.append(Region(line_v3, line_v4, line_h9, line_h10, 28))

regions.append(Region(line_v1, line_v2, line_h10, line_h11, 29))
regions.append(Region(line_v2, line_v3, line_h10, line_h11, 30))
regions.append(Region(line_v3, line_v4, line_h10, line_h11, 31))



In [40]:
def region_for_point(regions, lat, long):
    point_test = Point(lat,long)
    # Testing purpose
    region_num = []
    for region in regions:
        if region.check_inside(point_test):
            region_num.append(region.get_num())
    if len(region_num) > 1:
        raise Exception('Multiple values for', region_num)
    else:
        if len(region_num) == 0:
            return np.nan
        else:
            return region_num[0]

In [41]:
start_region_list = []
for i in range(data.shape[0]):
    start_region_list.append(region_for_point(regions, data['start station latitude'].iloc[i], data['start station longitude'].iloc[i]))

In [42]:
end_region_list = []
for i in range(data.shape[0]):
    end_region_list.append(region_for_point(regions, data['end station latitude'].iloc[i], data['end station longitude'].iloc[i]))

In [43]:
start_region_list_eval = []
for i in range(data_eval.shape[0]):
    start_region_list_eval.append(region_for_point(regions, data_eval['start station latitude'].iloc[i], data_eval['start station longitude'].iloc[i]))

In [44]:
end_region_list_eval = []
for i in range(data_eval.shape[0]):
    end_region_list_eval.append(region_for_point(regions, data_eval['end station latitude'].iloc[i], data_eval['end station longitude'].iloc[i]))

In [45]:
start_region_list_test = []
for i in range(data_test.shape[0]):
    start_region_list_test.append(region_for_point(regions, data_test['start station latitude'].iloc[i], data_test['start station longitude'].iloc[i]))

In [46]:
end_region_list_test = []
for i in range(data_test.shape[0]):
    end_region_list_test.append(region_for_point(regions, data_test['end station latitude'].iloc[i], data_test['end station longitude'].iloc[i]))

In [47]:
data['start region'] = start_region_list
data['end region'] = end_region_list
data_eval['start region'] = start_region_list_eval
data_eval['end region'] = end_region_list_eval
data_test['start region'] = start_region_list_test
data_test['end region'] = end_region_list_test

In [48]:
data.to_csv('data_train.csv')
data_eval.to_csv('data_val.csv')
data_test.to_csv('data_test.csv')